In [6]:

# import zipfile

# # Ruta del archivo ZIP
# zip_file_path = '/content/tesis-main.zip'

# # Ruta donde quieres extraer los archivos
# extract_to_path = '/content/'

# # Descomprimir el archivo
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_to_path)

# print("Archivos extraídos con éxito.")

In [7]:
# import sys
# sys.path.append('/content/tesis-main/')

from dotenv import load_dotenv

load_dotenv()

True

In [8]:

# import drive as drive_downloader
# drive_downloader.download_files('14_UUqgoWDPPNwXvdFjDoJomHTu5l85l6')

* Carga el video utilizando cv2.VideoCapture.
* Extrae los frames del video uno por uno.
* Preprocesa cada frame para que sean compatibles con el modelo de PyTorch (tamaño, normalización, etc.).
* Usa un modelo preentrenado de PyTorch (como ResNet) para extraer características de cada frame.
* Aplica un algoritmo de clustering (como K-Means) a las características extraídas.
* Asigna cada frame al cluster correspondiente.
* Visualiza los resultados mostrando los frames y su cluster correspondiente.

In [9]:
import torch
from sklearn.cluster import AgglomerativeClustering
import warnings
from preprocessing.transforms import BASELINE
from clustering.metrics import eval_massive_cluster
from experiment_framework import experiment
from validation import VALIDATION_DATASET
from utils import load_dataset
warnings.filterwarnings('ignore')
import wandb

torch.manual_seed(47)
import random
random.seed(47)
import numpy as np
np.random.seed(47)

if torch.cuda.is_available():
 dev = "cuda:0"
else:
 dev = "cpu"


device = torch.device(dev)
epochs = 10
steps=1
clustering_eval_function = lambda cluster_labels, tags: eval_massive_cluster(cluster_labels, tags, steps, epochs)

dataset_loader = load_dataset(VALIDATION_DATASET.keys())

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [11]:
from properties import MODELS_ENUM, PREPROCESSORS, ALGORITHM, GROUPER_FUNCTIONS

DEFAULT_PARAMS = {
      "dataset": dataset_loader,
      "evaluation_function": clustering_eval_function,
      "show": True,
      "device": device
   }


EXPERIMENT_DEFINITIONS= [
    {
        'name': 'ResnetAglomerative',
        'model': {
          'name': 'YOLOV8',
          'params': ['/Users/pio/postgrado/tesis/tesis/best.pt']
        },
        'algorithm': "AGGLOMERATIVE",
        'params': { "n_clusters":None, "distance_threshold": 2.2,"compute_distances":True},
        'grouper_function': "CLUSTERING_FUNCTION",
        'preprocessing': "YOLO_BASELINE",
    },
    {
        'name': 'SimilarityClustering',
        'model': {
          'name': "RESNET18",
          'params': [device]
        },
        'algorithm': "SIMILARITY",
        'params': {"threshold": 0.999},
        'grouper_function': "CLUSTERING_FUNCTION",
        'preprocessing': "BASELINE",
    }
]

for exp_definition in EXPERIMENT_DEFINITIONS:
  params = DEFAULT_PARAMS.copy()
  model_definition = MODELS_ENUM[exp_definition["model"]["name"]]
  model_params = exp_definition["model"]["params"]
  model = model_definition.get_instance(*model_params)
  params["model"] = model
  
  params["preprocessing"] = PREPROCESSORS[exp_definition["preprocessing"]]
  algorithm = ALGORITHM[exp_definition["algorithm"]](**exp_definition["params"])
  grouper_function = GROUPER_FUNCTIONS[exp_definition["grouper_function"]](algorithm)
  params["grouper_function"] = grouper_function
  params["name"] = exp_definition["name"]
  params["dataset"] = dataset_loader()

  run = wandb.init(project="ort-tesis",group=exp_definition['name'],config=exp_definition, entity="hortensias")
  result = experiment(**params)
  print(result._asdict())
  run.log({
    "Precision": result.precision.mean,
    "Accuracy": result.accuracy.mean,
    "F1": result.f1.mean,
    "Recall": result.recall.mean,
    "Precision_STD": result.precision.std,
    "Accuracy_STD": result.accuracy.std,
    "F1_STD": result.f1.std,
    "Recall_STD": result.recall.std,
  })
  run.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


[          0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0
           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0
           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0           0
           0           0           0           0           0 

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Accuracy,▁
Accuracy_STD,▁
F1,▁
F1_STD,▁
Precision,▁
Precision_STD,▁
Recall,▁
Recall_STD,▁
Accuracy,0.74581
Accuracy_STD,0.0
F1,0.73411


AttributeError: 'SimilarityClustering' object has no attribute 'distances_'